<a href="https://colab.research.google.com/github/SherbyRobotics/pyro/blob/colab/examples/notebooks/pyro_introduction_manipulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyro

## An object-based toolbox for robot dynamic simulation, analysis, control and planning


This page is an introduction to the basic functionnality of pyro, an open-source python library developped at Université de Sherbrooke and hosted here: https://github.com/SherbyRobotics/pyro

## Tutorial content

1.   [The Dynamic System class and basic functionnality](https://colab.research.google.com/drive/18eEL-n-dv9JZz732nFCMtqMThDcfD2Pr?usp=sharing)
2.   [Creating a custom dynamic class](https://colab.research.google.com/drive/1ILfRpL1zgiQZBOxwtbbpe0nl2znvzdWl?usp=sharing)
3.   [Closed-loop system and controllers objects](https://colab.research.google.com/drive/1mog1HAFN2NFEdw6sPudzW2OaTk_li0Vx?usp=sharing)
4.   The Linear System class (comin soon..)
4.   The Mechanical System class (coming soon..)
5.   **The Manipulator Robot class (this page)**





## Importing Pyro

This code show a quick exemple of

1.   Cloning pyro code source from the github repository
2.   Adding pyro folder to the python path
3.   Importing the library to the python interpreter



In [ ]:
!git clone https://github.com/SherbyRobotics/pyro
import sys
sys.path.append('/content/pyro')
import pyro


Here we import other basic python tools:

*   Numpy: the python library for linear algebra, on top of which pyro is built. 
*   Display: that is needed to show animation in the colab environment. If pyro is used locally then this is not needed.
*   Inspect: that we will use here only for printing source code in for this tutorial



In [ ]:
import numpy as np
from IPython import display
import inspect

# The Manipulator Robot Class

This class of system can be used to represent dynamic system (typically robotic arms) that can be described by the following second-order differential equation:

$H(q)\ddot{q} + C(\dot{q},q)\dot{q} + d(\dot{q},q) + g(q) = B(q) \tau + J(q)^T f_e $

where :

*   $H(q)$ is an inertia matrix
*   $C(\dot{q},q)$ is a matrix caracterizing coriolis and centrifugal effects
*   $B(q)$ is a matrix describing how the actuator act on the system
*   $J(q)$ is the jacobian matrix of the manipulator such that $J = \frac{\partial r}{\partial q}$
*   $d(\dot{q},q)$ is a vector describing the state-dependent dissipative forces
*   $g(q)$ is a vector describing the state-dependent conervative forces
*   $q$ is a vector of configuration variable describing the joint-space
*   $r$ is a vector of configuration variable describing the task-space (typically x-y-z of the tool at the end-effector)
*   $\tau$ is a vector of force\torque describing effort at the actuators 
*   $f_e$ is a vector of external forces at the end-effector

Furthermore, manipulator also implement kinematic relationship: a forward kinematic function that compute end-effector coordinates given joint-space variables:

$r = $forward_kinematic_effector$(q)$
    
and a forward differential kinematic function that compute end-effector velocities given joint-space velocities:

$\dot{r} = J(q) \dot{q}$

all manipulator object must implement methods defining the listing state-dependent properties, i.e. function defining matrix and vector that are functions of $q$ and $\dot{q}$.

## Exemple

Here we load as an exemple a class implementing a two link manipulator:

In [ ]:
from pyro.dynamic import manipulator

sys = manipulator.TwoLinkManipulator()

Now lets look under the hood to see the source code of those function implementing the dynamics and kinematic:

In [ ]:
print( inspect.getsource( sys.H ))

In [ ]:
print( inspect.getsource( sys.C ))

In [ ]:
print( inspect.getsource( sys.d ))

In [ ]:
print( inspect.getsource( sys.g ))

In [ ]:
print( inspect.getsource( sys.B ))

In [ ]:
print( inspect.getsource( sys.J ))

In [ ]:
print( inspect.getsource( sys.forward_kinematic_effector ))

Based on those definition, the more generic from of the dynamics:

$\dot{x} = f(x,u,t)$ \\
is automatically computed, and all generic analysis tools can be used.


In [ ]:
sys.x0   = np.array([ 0.1, 0.1, 0.0, 0.0]) # robot initial states [ joint 1 angle (rad),  joint 2 angle (rad), joint 1 velocity (rad/sec),  joint 1 velocity (rad/sec)]
sys.ubar = np.array([ 1.5, 0.5])           # robot default constant torques

# run the simulation
sys.compute_trajectory( tf = 6 )

# Animate and display the simulation
#sys.animate_simulation() # This would work locally in a python console
ani  = sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)

# Speed-controlled Manipulator Robot Class

This class of system can be used to represent robot arm behavior when the actuators take velocity commands. This model assume that low-level velocity controllers in the joints are perfect. The dynamics is reduced to 

$\dot{q} = u$

but with the following kinematic relationships: a forward kinematic function that compute end-effector coordinates given joint-space variables:

$r = $forward_kinematic_effector$(q)$
    
and a forward differential kinematic function that compute end-effector velocities given joint-space velocities:

$\dot{r} = J(q) \dot{q}$

where :

*   $J(q)$ is the jacobian matrix of the manipulator such that $J = \frac{\partial r}{\partial q}$
*   $q$ is a vector of configuration variable describing the joint-space
*   $r$ is a vector of configuration variable describing the task-space (typically x-y-z of the tool at the end-effector)
*   $u$ is a vector of velocity commands sent to the actuators

Manipulator object can be converted to speed-controlled manipulator using the following command:

In [ ]:
# Kinematic only model (inputs are motor velocities)
new_sys  = manipulator.SpeedControlledManipulator.from_manipulator( sys )

## Exemple

Here we run a simulation of the speed-controlled version of the same manipulator:

In [ ]:
new_sys.x0   = np.array([ 0.1, 0.1]) # robot initial states [ joint 1 angle (rad)]
new_sys.ubar = np.array([ 1.5, 0.5])     # robot default constant torques

# run the simulation
new_sys.compute_trajectory( tf = 6 )

# Animate and display the simulation
#new_sys.animate_simulation() # This would work locally in a python console
ani  = new_sys.generate_simulation_html_video()
html = display.HTML( ani )
display.display(html)